In [1]:
from agno.models import groq
import os
from dotenv import load_dotenv
import re

In [2]:
def nomorPutusan(content: str) -> str:
    """
    Mengekstrak Nomor Putusan dari teks dokumen.
    Args:
        content (str): Konten teks yang berisi nomor putusan.
    Returns:
        str: Nomor putusan yang ditemukan, atau None jika tidak ditemukan.
    """
    pattern = re.compile(
        r"P\s+U\s+T\s+U\s+S\s+A\s+N\s+(?:\n\s*)?Nomor\s+(\d{1,3}/Pid\.(?:B|Sus)/\d{4}/PN\s+\w+)",
        re.MULTILINE
    )
    match = pattern.search(content)
    if match:
        return match.group(1)
    return None

In [3]:
def dataTerdakwa(content: str) -> list:
    """
    Mengekstrak Informasi terdakwa dan return sebagai list of dictionaries.
    Setiap dictionary berisi:
        - Nama
        - Tempat Lahir
        - Umur (tanggal lahir)
        - Jenis Kelamin
        - Tempat tinggal
        - Agama
        - Pekerjaan

    Args:
        content (str): Konten teks yang berisi informasi terdakwa.
    Returns:
        list: List of dictionaries yang berisi informasi terdakwa.
    """
    terdakwa_pattern = re.compile(
        r"(?:T\s*erdakwa\s*\d*\s*\d*\.\s*)?"
        r"(?:\d+\.\s*)?Nama\s*lengkap\s*:?\s*(?P<nama_lengkap>[^;]+);\s*"
        r"(?:\d+\.\s*)?Tempat\s*lahir\s*:?\s*(?P<tempat_lahir>[^;]+);\s*"
        r"(?:\d+\.\s*)?Umur/Tanggal\s*lahir\s*:?\s*(?P<umur>[^;]+);\s*"
        r"(?:\d+\.\s*)?Jenis\s*kelamin\s*:?\s*(?P<jenis_kelamin>[^;]+);\s*"
        r"(?:\d+\.\s*)?Kebangsaan\s*:?\s*(?P<kebangsaan>[^;]+);\s*"
        r"(?:\d+\.\s*)?Tempat\s*tinggal\s*:?\s*(?P<tempat_tinggal>[^;]+);\s*"
        r"(?:\d+\.\s*)?Agama\s*:?\s*(?P<agama>[^;]+);\s*"
        r"(?:\d+\.\s*)?Pekerjaan\s*:?\s*(?P<pekerjaan>[^;]+)[.;]",
        re.IGNORECASE
    )

    results = []
    for match in terdakwa_pattern.finditer(content):
        data = match.groupdict()
        # Bersihkan nilai dengan strip()
        cleaned_data = {key: value.strip() for key, value in data.items()}
        results.append(cleaned_data)

    return results

In [4]:
def tuntutanPidana(content: str) -> str:
    """
    Mengekstrak informasi tuntutan pidana dari dokumen.
    Args:
        content (str): Konten teks yang berisi informasi tuntutan pidana.
    Returns:
        str: Teks yang berisi informasi tuntutan pidana.
    """
    pattern = re.compile(
        r"(?:T\s*untutan\s*pidana\s*\d*\s*\d*\.\s*)?"
        r"(?:\d+\.\s*)?Tuntutan\s*pidana\s*:?\s*(?P<tuntutan_pidana>[^;]+);",
        re.IGNORECASE
    )

    results = []
    for match in pattern.finditer(content):
        data = match.groupdict()
        cleaned_data = {key: value.strip() for key, value in data.items()}
        results.append(cleaned_data)

    return results

In [5]:
def normalizedPasal(pasal: str) -> str:
    """
    Normalisasi teks pasal untuk konsistensi.
    Menghapus spasi berlebih, mengubah ke huruf kecil,
    dan mengganti istilah tertentu.
    Args:
        pasal (str): Teks pasal yang akan dinormalisasi.
    Returns:
        str: Teks pasal yang telah dinormalisasi.
    """
    pasal = re.sub(r'\s+', ' ', pasal.lower())
    pasal = pasal.replace('kitab undang - undang hukum pidana', 'kuhp')
    pasal = pasal.replace('kitab undang-undang hukum pidana', 'kuhp')
    pasal = pasal.replace('jo.', 'jo')
    pasal = pasal.replace('junctor', 'jo')
    return pasal.strip()

def relatedPasal(content: str) -> list:
    """
    Mengekstrak pasal terkait dari teks dokumen.
    Menggunakan regex untuk menemukan pola pasal, ayat, dan referensi Jo.
    Mengembalikan daftar pasal yang telah dinormalisasi.
    Args:
        content (str): Konten teks yang berisi informasi pasal.
    Returns:
        list: Daftar pasal yang telah dinormalisasi.
    """
    pattern = r'''(?ix)
        pasal\s+\d+                                   # Pasal 378
        (?:\s+ayat\s+\(?\d+\)?                        # ayat (1)
            (?:\s*,\s*ayat\s+\(?\d+\)?)*              # , ayat (2)
            (?:\s*(?:dan|,)\s*ayat\s+\(?\d+\)?)?      # dan ayat (3)
        )?
        (?:\s+ke-?\d+)?                               # ke-1
        (?:\s+UU\s+RI\s+No\.\s*\d+\s*Tahun\s*\d+)?    # UU RI No. 17 Tahun 2023
        (?:\s+(?:KUHP|Kitab\s+Undang[\s-]*Undang\s+Hukum\s+Pidana))?
        (?:
            \s+(?:Jo\.?|Juncto)\s+Pasal\s+\d+         # Jo. Pasal 55
            (?:\s+ayat\s+\(?\d+\)?                    # ayat (1)
                (?:\s*,\s*ayat\s+\(?\d+\)?)*          # , ayat (2)
                (?:\s*(?:dan|,)\s*ayat\s+\(?\d+\)?)?  # dan ayat (3)
            )?
            (?:\s+ke-?\d+)?                           # ke-1
            (?:\s+(?:KUHP|Kitab\s+Undang[\s-]*Undang\s+Hukum\s+Pidana))?
        )*
    '''
    matches = re.findall(pattern, content, re.MULTILINE)
    return list(dict.fromkeys(m.strip() for m in matches if m.strip()))

In [6]:
def tuntutanHukuman(content: str) -> str:
    """
    Mengekstrak durasi tuntutan hukuman dari teks dokumen.
    Mencari frasa yang mengandung "dengan hukuman penjara" dan mengambil durasi setelahnya.
    Contoh durasi yang diharapkan: "5 (lima) tahun", "10 tahun 2 bulan", "8 bulan".

    Args:
        content (str): Konten teks yang berisi informasi tuntutan hukuman.
    Returns:
        str: Durasi tuntutan hukuman yang ditemukan, atau None jika tidak ditemukan.
    """
    block = re.search(r'pembacaan tuntutan pidana.*?(?=\n\n|\Z)', content, re.IGNORECASE | re.DOTALL)
    if not block:
        return None

    # Ambil frasa durasi setelah "dengan pidana penjara"
    match = re.search(
        r'dengan pidana penjara\s+(.*?)(?:[.,;\n]|dikurangi|dengan perintah)',
        block.group(0),
        re.IGNORECASE | re.DOTALL
    )

    if match:
        durasi = match.group(1).strip()
        return durasi  # contoh: "5 (lima) tahun", "10 tahun 2 bulan", "8 bulan"
    return None


In [7]:
def putusanPidana(content: str) -> str:
    """
    Mengekstrak durasi putusan pidana dari teks dokumen.
    Mencari frasa yang mengandung "Menjatuhkan pidana" dan mengambil durasi setelahnya.
    Contoh durasi yang diharapkan: "4 (empat) tahun", "5 tahun 6 bulan", "3 bulan".
    Args:
        content (str): Konten teks yang berisi informasi putusan pidana.
    Returns:
        str: Durasi putusan pidana yang ditemukan, atau None jika tidak ditemukan.
    """

    block = re.search(r'MENGADILI:(.*?)(?=\n\n|\Z)', content, re.IGNORECASE | re.DOTALL)
    if not block:
        return None

    match = re.search(
        r'Menjatuhkan pidana.*?dengan pidana penjara\s+selama\s+(.*?)(?:[.,;\n]|dan|dikurangi)',
        block.group(1),
        re.IGNORECASE | re.DOTALL
    )

    if match:
        durasi = match.group(1).strip()
        return durasi  # contoh: "4 (empat) tahun", "5 tahun 6 bulan"
    return None


In [8]:
def putusanHukuman(content: str) -> str:
    """
    Mengekstrak informasi putusan hukuman dari teks dokumen.
    Mencari blok teks yang dimulai dengan "MENGADILI:" dan berakhir sebelum "Demikianlah diputuskan".
    Args:
        content (str): Konten teks yang berisi informasi putusan hukuman.
    Returns:
        str: Teks putusan hukuman yang ditemukan, atau None jika tidak ditemukan.
    """
    match = re.search(r'MENGADILI:(.*?)(?=Demikianlah diputuskan)', content, re.IGNORECASE | re.DOTALL)
    if match:
        return match.group(1).strip()
    return None


In [9]:
def tanggalPutusan(content: str) -> str:
    """
    Mengekstrak tanggal putusan dari dokumen.
    """
    match = re.search(r"Demikianlah\s+diputuskan.*?pada\s+hari\s+(.+?)(?:,\s*)?oleh\s+kami", content, re.IGNORECASE | re.DOTALL)
    if match:
        tanggal_putusan = match.group(1).strip()

    return tanggal_putusan


In [10]:
def get_entity_with_re(file_path: str) -> str:
    """
    Extracts the entity name from a file path using a regular expression.
    The function reads the content of the file, applies a regex pattern to find
    the entity name, and returns the matched entity.
    Args:
    file_path (str): The path of txt file file.

    Returns:
    str: The matched entity name.
    """
    entities = {
        "Nomor Putusan": None,
        "Tanggal Putusan": None,
        "Informasi Terdakwa": None,
        "Related Pasal": None,
        "Tuntutan Pidana": None,
        "Tuntutan Hukuman": None,
        "Putusan Pidana": None,
        "Putusan Hukuman": None,
        "Hakim Ketua": None,
        "Hakim Anggota": None,
        "Penuntut Umum": None,
    }
    with open(file_path, "r", encoding="utf-8", errors="replace") as file:
        content = file.read()

    nomor_putusan = nomorPutusan(content)
    if nomor_putusan:
        entities["Nomor Putusan"] = nomor_putusan

    tanggal_putusan = tanggalPutusan(content)
    if tanggal_putusan:
        entities["Tanggal Putusan"] = tanggal_putusan    
    
    data_terdakwa = dataTerdakwa(content)
    if data_terdakwa:
        entities["Informasi Terdakwa"] = data_terdakwa
    
    tuntutan_pidana = tuntutanPidana(content)
    if tuntutan_pidana:
        entities["Tuntutan Pidana"] = tuntutan_pidana[0]["tuntutan_pidana"]

    related_pasal_matches = relatedPasal(content)
    if related_pasal_matches:
        entities["Related Pasal"] = related_pasal_matches
    
    tuntutan_hukuman = tuntutanHukuman(content)
    if tuntutan_hukuman is not None:
        entities["Tuntutan Hukuman"] = tuntutan_hukuman

    putusan_pidana = putusanPidana(content)
    if putusan_pidana is not None:
        entities["Putusan Pidana"] = putusan_pidana    

    putusan_hukuman = putusanHukuman(content)
    if putusan_hukuman:
        entities["Putusan Hukuman"] = putusan_hukuman
    return entities

In [11]:
get_entity_with_re("samples/171.Pid.Sus.2024.PN.Gpr.txt")

{'Nomor Putusan': '171/Pid.Sus/2024/PN Gpr',
 'Tanggal Putusan': 'Kamis tanggal 19 September 2024',
 'Informasi Terdakwa': [{'nama_lengkap': 'RIZAL ADITIYAWAN BASUKI UTOMO BIN Alm. NANANG BASUKI',
   'tempat_lahir': 'Kediri',
   'umur': '20 tahun/ 27 Mei 2003',
   'jenis_kelamin': 'Laki-laki',
   'kebangsaan': 'Indonesia',
   'tempat_tinggal': 'Jl.Letjen Hariyono 39D, RT.034/RW.007, Kel/Desa Singonegaran, Kec. Pesantren, Kota Kediri, Provinsi Jawa Timur',
   'agama': 'Islam',
   'pekerjaan': 'Swasta'}],
 'Related Pasal': ['Pasal 60 ke-10 Juncto Pasal 106 ayat (1) dan ayat (2)',
  'Pasal 435 Juncto Pasal 138 ayat (2) dan ayat (3)',
  'Pasal 436 ayat (2) Juncto Pasal 145 ayat (1)',
  'Pasal 138 ayat (2) dan ayat (3)',
  'pasal 1',
  'Pasal 320 UU RI No. 17 Tahun 2023',
  'Pasal 6 ayat 1, ayat 2 dan ayat 3 Jo. Pasal 2 ayat 1'],
 'Tuntutan Pidana': 'yang diajukan oleh Penuntut Umum yang pada pokoknya sebagai berikut:\n1.Menyatakan Terdakwa RIZAL ADITIYAWAN BASUKI UTOMO BIN (ALM) NANANG BAS